In [1]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator

In [2]:
def train_test_split(X, ratio=0.2, user_col='reviewerID', item_col='asin',
                     rating_col='overall', time_col='unixReviewTime'):
    # сортируем оценки по времени
    X.sort_values(by=[time_col], inplace=True)
    # список всех юзеров
    userIds = X[user_col].unique()
    X_train_data = []
    X_test_data = []
    y_train = []
    y_test = []
    for userId in tqdm(userIds):
        curUser = X[X[user_col] == userId]
        # определяем позицию, по которой делим выборку и размещаем данные по массивам
        idx = int(curUser.shape[0] * (1 - ratio))
        X_train_data.append(curUser[[user_col, item_col]].iloc[:idx, :].values)
        X_test_data.append(curUser[[user_col, item_col]].iloc[idx:, :].values)
        y_train.append(curUser[rating_col].values[:idx])
        y_test.append(curUser[rating_col].values[idx:])
    # cтекуем данные по каждому пользователю в общие массивы
    X_train = pd.DataFrame(np.vstack(X_train_data), columns=[user_col, item_col])
    X_test = pd.DataFrame(np.vstack(X_test_data), columns=[user_col, item_col])
    y_train = np.hstack(y_train)
    y_test = np.hstack(y_test)
    return X_train, X_test, y_train, y_test

In [3]:
df = pd.read_json('Digital_Music_5.json', lines=True)

In [4]:
df[['overall', 'reviewerID', 'unixReviewTime']]

,overall,reviewerID,unixReviewTime
0,5,A2TYZ821XXK2YZ,1370217600
1,5,A3OFSREZADFUDY,1412985600
2,5,A2VAMODP8M77NG,1392076800
3,4,AAKSLZ9IDTEH0,1386374400
4,5,A3OH43OZJLKI09,1465689600
...,...,...,...
169776,5,A1SR2T84IXOMAQ,1534636800
169777,5,A2SR3DWJR1PYR6,1521331200
169778,5,A24V7X30NIMOIY,1520380800
169779,5,A1LW10GYP2EYM1,1518393600


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df)

100%|██████████| 16566/16566 [07:50<00:00, 35.22it/s]
